# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI.

In [3]:
import wikipedia
import numpy as np
from nltk import sent_tokenize
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

c:\Users\aomak\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def load_with_disambigution(page):
    try:
        p = wikipedia.page(page)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        p = wikipedia.page(random_option)
    return p

def get_texts_for_lang(lang, n=10): 
    wikipedia.set_lang(lang)
    wiki_content = []
    
    pages = 0
    
    while pages < 10:
        try:
            page_name = wikipedia.random(1)
            page = load_with_disambigution(page_name)
            pages += 1
        
        except Exception as e:
            continue

        wiki_content.append(f'{page.title}\n{page.content.replace("==", "")}')

    return wiki_content

In [5]:
langs = ['de', 'fr', 'ro', 'it', 'hr']

wiki_texts = {}

for lang in langs:
    try:
        wiki_texts[lang] = get_texts_for_lang(lang)
    except Exception as e:
        print('ERROR ON - ', lang, e)
        continue
    
    print(lang, len(wiki_texts[lang]))

c:\Users\aomak\AppData\Local\Programs\Python\Python37\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\aomak\AppData\Local\Programs\Python\Python37\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


de 10
fr 10
ro 10
it 10
hr 10


In [15]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import re
from string import digits
from collections import defaultdict

In [13]:
def predict_language(text, lang2char):
    text_chars = set(re.findall('\w', text.lower())) - digits
    lang2sim = {}
    
    for lang in lang2char:
        intersect = len(text_chars & lang2char[lang])
        lang2sim[lang] = intersect
    
    return max(lang2sim.items(), key=lambda x: x[1])[0]

In [19]:
all_sentences = {}
for lang in wiki_texts:
    for article in wiki_texts[lang]:
        sentences = sent_tokenize(article)
        for sent in sentences:
            sent.replace(r'=+', '')
            sent.replace(r'\n\n+?', '')
            all_sentences[sent] = lang

digits = set(digits)

lang2chars = defaultdict(set)

for lang in wiki_texts:
    for text in wiki_texts[lang]:
        char_set = set(re.findall("\w", text.lower())) - digits
        lang2chars[lang].update(char_set)

true_labels = []
predicted_labels = []

for lang in wiki_texts:
    for text in wiki_texts[lang]:
        true_labels.append(lang)
        predicted_labels.append(predict_language(text, lang2chars))

ARI = []

X = np.zeros((len(all_sentences), 768))

for i, text in enumerate(all_sentences.keys()):
    X[i] = embed(text)

cluster = KMeans(3)

cluster.fit(X)
labels = predicted_labels

ARI.append(adjusted_rand_score(true_labels, predicted_labels))
    
print(np.mean(ARI))

0.8969106411250138


# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

In [83]:
import bz2

with bz2.open('annot.opcorpora.no_ambig_strict.xml.bz2', 'rb') as f_in, open('annot.opcorpora.no_ambig_strict.xml', 'wb') as f_out:
    f_out.write(f_in.read())

In [84]:
from lxml import etree
from itertools import islice

open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

dataset = {}

for sentence in open_corpora.xpath('//tokens'):
    for token in sentence.xpath('token'):
        word = token.xpath('@text')[0].lower()
        gram_info = token.xpath('tfr/v/l/g/@v')
        if word not in dataset:
            dataset[word] = [0, 0, []]
        if gram_info not in dataset[word][2]:
            dataset[word][1] += 1
            dataset[word][2].append(gram_info)
        dataset[word][0] += 1
        
lots_of_graminfo = {k: v[1] for k, v in dataset.items() if v[1] > 4}
print('Словоформы с наибольшим числом разборов:')
print(lots_of_graminfo)

most_frequent = {k: v[0] for k, v in sorted(dataset.items(), key=lambda item: item[1], reverse=True) if v[1] > 1}
print('Самые частотные омонимичные словоформы:')
print(list(islice(most_frequent.items(), 10)))


Словоформы с наибольшим числом разборов:
{'кино': 5, 'евро': 5, 'сша': 6, 'компании': 5, 'пути': 5, 'какой': 5}
Самые частотные омонимичные словоформы:
[('в', 2059), ('на', 786), ('с', 613), ('и', 574), ('о', 213), ('году', 115), ('а', 113), ('этом', 104), ('россии', 91), ('было', 89)]


## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуру через DependencyGraph, выберите отношение (из тех, что не упоминались на семинаре) и найдите самые частотные пары слов, связанных этим отношением. 

Для самой частотной пары вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)`) В итоге у вас должен получится список пары: слово1 и все его подзависимые - слово2 и все его подзависимые.

Визуализируйте самое длинное предложение через networkx

In [22]:
import spacy_udpipe

TypeError: Plain typing.NoReturn is not valid as type argument